# Bibliotecas / Conectar o Drive

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install --upgrade geopandas
import geopandas as gpd

     |████████████████████████████████| 1.0MB 4.3MB/s 
     |████████████████████████████████| 6.5MB 34.7MB/s 
     |████████████████████████████████| 15.3MB 204kB/s 


In [ ]:
pip install plotly==4.9.0

     |████████████████████████████████| 12.9MB 4.1MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import plotly
print(plotly.__version__)

4.9.0


In [ ]:
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Mapas Dinâmicos: Casos e óbitos ao longo do tempo

## Lendo Arquivos e preparando informações geográficas para o mapa

In [ ]:
df_ibge_estados = pd.read_csv("/content/drive/MyDrive/VisaVi-SE/Conteúdo do Portal/Mapas Dinâmicos: Casos e óbitos ao longo do tempo/db_ibge_estados.csv")
df_ibge_municipios = pd.read_csv("/content/drive/MyDrive/VisaVi-SE/Conteúdo do Portal/Mapas Dinâmicos: Casos e óbitos ao longo do tempo/db_ibge_municipios.csv")
gp_sergipe_geojson = gpd.read_file("/content/drive/MyDrive/VisaVi-SE/Conteúdo do Portal/Mapas Dinâmicos: Casos e óbitos ao longo do tempo/json_geojs-sergipe.json", driver='GeoJSON')

In [ ]:
# Pega os valores de latitude e longitude do IBGE de sergipe
latitude_ibge_estado_sergipe = df_ibge_estados[df_ibge_estados['uf'] == 'SE']['latitude'].iloc[0]
longitude_ibge_estado_sergipe = df_ibge_estados[df_ibge_estados['uf'] == 'SE']['longitude'].iloc[0]

# Pegar os nomes de todos municipios de sergipe pelo IBGE
colunas_ibge_municipios_sergipe = ['nome']
db_ibge_municipios_sergipe = df_ibge_municipios[df_ibge_municipios['codigo_uf'] == 28]
db_ibge_municipios_sergipe = db_ibge_municipios_sergipe.filter(items=colunas_ibge_municipios_sergipe)
db_ibge_municipios_sergipe.rename(columns={'nome': 'municipio'}, inplace = True)
db_ibge_municipios_sergipe.reset_index(drop=True, inplace=True)

## Lendo e Organizando o DataFrame

In [ ]:
caso_0 = pd.read_csv("/content/drive/MyDrive/VisaVi-SE/Conteúdo do Portal/Mapas Dinâmicos: Casos e óbitos ao longo do tempo/caso.csv")
df_caos_obitos = caso_0.copy()

In [ ]:
dados_sergipe = df_caos_obitos.loc[df_caos_obitos['state'] == "SE"]
dados_sergipe = dados_sergipe.filter(items=['date', 'city_ibge_code', 'city', 'confirmed', 'deaths'])
dados_sergipe = dados_sergipe.sort_values(['city','date'])

In [ ]:
dados_sergipe.head()

,date,city_ibge_code,city,confirmed,deaths
819607,2020-05-10,2800100.0,Amparo de São Francisco,1,0
819606,2020-05-11,2800100.0,Amparo de São Francisco,1,0
819605,2020-05-12,2800100.0,Amparo de São Francisco,1,0
819604,2020-05-13,2800100.0,Amparo de São Francisco,2,0
819603,2020-05-14,2800100.0,Amparo de São Francisco,2,0


In [ ]:
dados_sergipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14567 entries, 819607 to 819179
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            14567 non-null  object 
 1   city_ibge_code  14567 non-null  float64
 2   city            14321 non-null  object 
 3   confirmed       14567 non-null  int64  
 4   deaths          14567 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 682.8+ KB


In [ ]:
# Passa a coluna  city_ibge_code para o tipo int

dados_sergipe['city_ibge_code'] = dados_sergipe['city_ibge_code'].astype(int)

In [ ]:
# Passa as datas para o tipo int

def NumeraData(data):
  data = data.replace("-", "")
  data = int(data[0:4] + data[4:6] + data[6:8])
  return data

dados_sergipe['date'] = dados_sergipe['date'].apply(lambda x: NumeraData(x))

### Removendo linhas onde os valores de cidades estão ausentes

In [ ]:
# Quantidade de nan presentes nas colunas

dados_sergipe.isnull().sum()

date                0
city_ibge_code      0
city              246
confirmed           0
deaths              0
dtype: int64

In [ ]:
# Verifica quantidade de dadas diferentes que estão no DataFrame

datas = pd.unique(dados_sergipe.date)
len(datas)

246

Existe uma faixa de datas inteira sem valor de cidade

In [ ]:
# Removo as linhas onde não existem dados de cidades

dados_sergipe.dropna(inplace=True)
dados_sergipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14321 entries, 819607 to 833552
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            14321 non-null  int64 
 1   city_ibge_code  14321 non-null  int64 
 2   city            14321 non-null  object
 3   confirmed       14321 non-null  int64 
 4   deaths          14321 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 671.3+ KB


In [ ]:
dados_sergipe

,date,city_ibge_code,city,confirmed,deaths
819607,20200510,2800100,Amparo de São Francisco,1,0
819606,20200511,2800100,Amparo de São Francisco,1,0
819605,20200512,2800100,Amparo de São Francisco,1,0
819604,20200513,2800100,Amparo de São Francisco,2,0
819603,20200514,2800100,Amparo de São Francisco,2,0
...,...,...,...,...,...
833556,20201118,2807600,Umbaúba,816,24
833555,20201120,2807600,Umbaúba,824,24
833554,20201122,2807600,Umbaúba,824,24
833553,20201123,2807600,Umbaúba,826,24


### Verificando se os nomes dos municípios no DataFrame são iguais aos nomes presentes no JSON

Quase todos os nomes das cidades estão corretos 

Usando o trecho abaixo para corrigir o nome errado o mapa pode ser montado pelo parâmetro de nome da cidade:

dados_sergipe['city'] = dados_sergipe['city'].apply(lambda x: str(x).replace("Graccho Cardoso", "Gracho Cardoso"))

Parâmetro na montagem do mapa:

featureidkey="properties.name"

In [ ]:
citys = pd.unique(dados_sergipe.city)
citys2 = pd.unique(gp_sergipe_geojson.name)

In [ ]:
len(citys) == len(citys2)

True

In [ ]:
citys.sort()
citys2.sort()
citys == citys2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
# Verifica qual nome de cidade está errado

for i in range(0, 75):
  if citys[i] != citys2[i]:
    print(citys[i], "---", citys2[i], i)

Graccho Cardoso --- Gracho Cardoso 23


In [ ]:
dados_sergipe['city'] = dados_sergipe['city'].apply(lambda x: str(x).replace("Graccho Cardoso", "Gracho Cardoso"))

### Verificando se os ids no DataFrame são iguais aos ids presentes no JSON

Todos os IDs estão corretos e podem ser utilizados para gerar o mapa no parâmetro:

featureidkey="properties.id"

In [ ]:
id = pd.unique(dados_sergipe.city_ibge_code)
id2 = pd.unique(gp_sergipe_geojson.id)

In [ ]:
len(id) == len(id2)

True

In [ ]:
id.sort()
id2.sort()
id == id2

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [ ]:
print(id[0], id2[0])

2800100 2800100


In [ ]:
type(id[0])

numpy.int64

In [ ]:
type(id2[0])

str

In [ ]:
# Passa a coluna city_ibge_code para o tipo string

dados_sergipe['city_ibge_code'] = dados_sergipe['city_ibge_code'].astype(str)

In [ ]:
id = pd.unique(dados_sergipe.city_ibge_code)
id.sort()
id == id2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

### Verificando as datas

In [ ]:
dados_sergipe.head()

,date,city_ibge_code,city,confirmed,deaths
819607,20200510,2800100,Amparo de São Francisco,1,0
819606,20200511,2800100,Amparo de São Francisco,1,0
819605,20200512,2800100,Amparo de São Francisco,1,0
819604,20200513,2800100,Amparo de São Francisco,2,0
819603,20200514,2800100,Amparo de São Francisco,2,0


In [ ]:
# Lista de datas presentes no DataFrame e lista com a quantidade de vezes que elas aparecem

dates, count_date = np.unique(dados_sergipe.date, return_counts=True)

In [ ]:
dates

array([20200314, 20200316, 20200318, 20200320, 20200321, 20200324,
       20200325, 20200327, 20200330, 20200331, 20200401, 20200402,
       20200403, 20200404, 20200406, 20200407, 20200408, 20200409,
       20200410, 20200411, 20200413, 20200414, 20200415, 20200416,
       20200417, 20200418, 20200419, 20200420, 20200421, 20200422,
       20200423, 20200424, 20200425, 20200426, 20200427, 20200428,
       20200430, 20200501, 20200502, 20200503, 20200504, 20200505,
       20200506, 20200507, 20200508, 20200509, 20200510, 20200511,
       20200512, 20200513, 20200514, 20200515, 20200516, 20200517,
       20200518, 20200519, 20200520, 20200521, 20200522, 20200523,
       20200524, 20200525, 20200526, 20200527, 20200528, 20200529,
       20200530, 20200531, 20200601, 20200602, 20200603, 20200604,
       20200605, 20200606, 20200607, 20200608, 20200609, 20200610,
       20200611, 20200612, 20200613, 20200614, 20200615, 20200616,
       20200617, 20200618, 20200619, 20200620, 20200621, 20200

In [ ]:
count_date

array([ 1,  1,  2,  2,  2,  3,  3,  3,  3,  3,  3,  4,  4,  5,  5,  6,  7,
        7,  7,  7,  7,  7,  9, 11, 11, 13, 14, 15, 15, 16, 16, 16, 17, 19,
       23, 26, 32, 34, 38, 42, 43, 45, 47, 48, 48, 53, 57, 58, 58, 58, 58,
       58, 63, 64, 66, 68, 70, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75

In [ ]:
# Separa todas as datas que aparecem 75 vezes (1 vez em cada município) e armazena na lista datas_gerais

datas_gerais = []

for i in range(0, len(dates)):
  if count_date[i] == 75:
    datas_gerais.append(dates[i])

### Filtrando o DataFrame com base nas datas gerais

In [ ]:
# Passo pelo DataFrame dados_sergipe e preencho as listas com os valores onde as datas pertecem a lista datas_gerais

date = []
city_ibge_code = []
city = []
confirmed = []
deaths = []
lista_index = dados_sergipe.index # Lista com os indices do DataFrame na ordem atual em que se encontram


for i in lista_index:
  if dados_sergipe['date'][i] in datas_gerais:
    date.append(dados_sergipe['date'][i])
    city_ibge_code.append(dados_sergipe['city_ibge_code'][i])
    city.append(dados_sergipe['city'][i])
    confirmed.append(dados_sergipe['confirmed'][i])
    deaths.append(dados_sergipe['deaths'][i])

In [ ]:
# Crio um novo DataFrame com os dados das listas feitas acima, com os dados filtrados pelas datas

dic_dados_filtrados = {'date': date,
                      'city_ibge_code': city_ibge_code,
                      'city': city,
                      'confirmed': confirmed,
                      'deaths': deaths}
dados_sergipe_filtrado = pd.DataFrame(data=dic_dados_filtrados)

## Exportando DataFrame

In [ ]:
dados_sergipe_filtrado.to_csv("/content/drive/MyDrive/VisaVi-SE/Conteúdo do Portal/Mapas Dinâmicos: Casos e óbitos ao longo do tempo/DF_Casos_Obitos_Tempo.csv")

## Mapa Dinâmico: Casos Confirmados ao longo do tempo

In [ ]:
dados_sergipe_filtrado

In [ ]:
dados_sergipe_filtrado.describe()

Usando: featureidkey="properties.id"

O id da cidade aparece no mapa ao posicionar o mouse em cima da região

Se utilizasse featureidkey="properties.name" o nome seria mostrado

In [ ]:
fig = px.choropleth_mapbox(dados_sergipe_filtrado, geojson=gp_sergipe_geojson, locations="city_ibge_code", color="confirmed",
                           featureidkey="properties.id", range_color=(0, 6000), animation_frame="date", mapbox_style="carto-positron",
                           center={'lat':latitude_ibge_estado_sergipe, 'lon':longitude_ibge_estado_sergipe},
                           zoom=8, height=800, color_continuous_scale='Inferno')

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Mapa Dinâmico: Número de Mortes ao longo do tempo

In [1]:
dados_sergipe_filtrado

NameError: ignored

In [ ]:
dados_sergipe_filtrado.describe()

Usando: featureidkey="properties.name"

O nome da cidade aparece no mapa ao posicionar o mouse em cima da região

In [ ]:
fig = px.choropleth_mapbox(dados_sergipe_filtrado, geojson=gp_sergipe_geojson, locations="city", color="deaths",
                           featureidkey="properties.name", range_color=(0, 200), animation_frame="date", mapbox_style="carto-positron",
                           center={'lat':latitude_ibge_estado_sergipe, 'lon':longitude_ibge_estado_sergipe},
                           zoom=8, height=800, color_continuous_scale='Inferno')

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()